In [4]:
# Import myfunc at cix folder
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../cix')
import myfuncs as mf
import pandas as pd
from rdkit import rdBase
rdBase.DisableLog('rdApp.*') # To make rdkit silent
from rdkit.Chem import PandasTools as pt
from rdkit.Chem import Descriptors
import numpy as np

In [5]:
it = [50000, 81661]
nit = len(it)

In [ ]:
df_un, cls = mf.wholean(it = it, name_train = "train", name_pref = "unc")

Clustering time: 00:03:25
Diversity analysis time: 00:06:39
Clustering time: 00:00:19
Diversity analysis time: 00:00:37
Arenas creation time: 00:03:36
Novelty analysis time: 00:07:00
Clustering time: 00:05:52


In [ ]:
df_co, clsco = mf.wholean(it = it, name_train = "train", name_pref = "con")